In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib
from collections import Counter
import sklearn as skl
import sklearn.cluster as skcluster
import scipy.stats

In [ ]:
!conda list

Read in Data

In [ ]:
OVCAR_dt=pd.read_csv("DESEQ_Normalized_Output_dt_nonzero_genes.tsv", sep="\t",header = 0)
OVCAR_dt[:10]

Process the data to define the mean of every gene and quartile information

In [ ]:
samplenames = []
for val in OVCAR_dt.columns:
    if val[0:3] == 'OVC':
        samplenames.append(val)
samplenames[:10]

In [ ]:
OVCAR_nozero = np.array(OVCAR_dt)

In [ ]:
genemean = []
for row in OVCAR_nozero[:,1:95]:
    means = np.mean(row)
    genemean.append(means)
OVCAR_dt['Gene_Mean'] = genemean

In [ ]:
lowerthreshold_TQN = []
upperthreshold_TQN = []
for row in OVCAR_nozero[:,1:95]:
    lowq = np.percentile(row,25)
    upq = np.percentile(row,75)
    IQR = upq - lowq
    lowthresh = lowq - (1.5 * IQR)
    upperthresh = upq + (1.5 * IQR)
    lowerthreshold_TQN.append(lowthresh)
    upperthreshold_TQN.append(upperthresh)
    
    
OVCAR_dt['LowerThreshold'] = lowerthreshold_TQN
OVCAR_dt['UpperThreshold'] = upperthreshold_TQN

OVCAR_dt[:10]

Rank all genes by thier mean expression across the entire cohort; from highest to lowest expression (Rank 1 = highest expression)

In [ ]:
OVCAR_dt['Rank'] = OVCAR_dt['Gene_Mean'].rank(method = 'average', ascending = False)

OVCAR_dt[:10]

In [ ]:
ranknames = []
for val in samplenames:
    name = val + '_rank'
    ranknames.append(name)
    OVCAR_dt[name] = OVCAR_dt[val].rank(method = 'average', ascending = False)
OVCAR_dt[:10]

Determine the change in rank value for every gene in every sample

In [ ]:
DevNames = []
Deviation_in_Rank = []
for val in ranknames:
    geneset = [val]
    newname = val+'_deviation_in_rank'
    DevNames.append(newname)
    for i in range(len(OVCAR_dt[val])):
        rank = OVCAR_dt[val][i]
        sample = val
        #median = DESEQ_dt['Median'][i]
        true_rank = OVCAR_dt['Rank'][i]
        gene = OVCAR_dt['Gene'][i]
        #genechrom = DESEQ_dt['Chromosomes'][i]
        rank_dif = true_rank - rank
        #OVCAR_dt[newname][i] = true_rank
        Deviation_in_Rank.append([sample, gene, rank_dif])
    

In [ ]:
for val in DevNames:
    set_of_values = []
    for i in range(len(Deviation_in_Rank)):
        name = val.split('_')[:3]
        name2 = Deviation_in_Rank[i][0].split('_')[:3]
        vals = Deviation_in_Rank[i][2]
        if name == name2:
            set_of_values.append(vals)
    OVCAR_dt[val] = set_of_values
OVCAR_dt[:10]

Establish Negative Controls

In [ ]:
ControlNames = ['OVCAR3_H10_DORM_1_no_chro_rank_deviation_in_rank','OVCAR3_G11_SCRAMBLE_1_no_chro_rank_deviation_in_rank']

In [ ]:
Dev_values_control = []
maxes = []
mins = []
for name in ControlNames:
    namevals = []
    for i in range(len(OVCAR_dt[name])):
        Dev_values_control.append(OVCAR_dt[name][i])
        namevals.append(OVCAR_dt[name][i])
    maxes.append(np.max(namevals))
    mins.append(np.min(namevals))
Dev_values_control[:3]
Dev_values_only = []
maxes = []
mins = []
for name in DevNames:
    namevals = []
    for i in range(len(OVCAR_dt[name])):
        Dev_values_only.append(OVCAR_dt[name][i])
        namevals.append(OVCAR_dt[name][i])
    maxes.append(np.max(namevals))
    mins.append(np.min(namevals))
Dev_values_only[:3]


I can now look at the overall genomic distribution of rank changes

# Rank Change Distribution Plot 

In [ ]:
matplotlib.pyplot.figure(figsize=(10,10))
sns.distplot(Dev_values_only,axlabel= 'Rank Order Changes')
#matplotlib.pyplot.savefig('Rank_Order_Distribution.pdf')

# FDR Function

In [ ]:
#FDR function that accepts lengths of gene lists from normals and non-normals 
#need to do sample by sample, should be the median of the controls vs the total count 
def FDR_Calculation(normals,rest):
    firstset = np.median(normals)
    values = []
    boolvals = []
    combo_of_vals = []
    #when looking for threshold cutoffs 
    FDR_Threshold_Internal = .05
    for i in range(len(rest)):
        secondset = rest[i]
        value = firstset/secondset
        values.append(value)
        booleanvalue = value < FDR_Threshold_Internal
        boolvals.append(booleanvalue)
        combo_of_vals.append([value,booleanvalue])
            
        
        
    return(values)

def FDR_Calculation_Threshold(normals,rest,thresh):
    firstset = np.median(normals)
    values = []
    boolvals = []
    combo_of_vals = []
    #when looking for threshold cutoffs 
    FDR_Threshold_Internal = thresh
    for i in range(len(rest)):
        name = FDR_Threshold_Up[i][0]
        secondset = rest[i]
        if secondset == 0:
            value = 'NA'
            values.append(value)
            booleanvalue = 'NA'
            boolvals.append(booleanvalue)
            combo_of_vals.append([value,booleanvalue,name,i])
        elif secondset != 0:
            value = firstset/secondset
            values.append(value)
            booleanvalue = value < FDR_Threshold_Internal
            boolvals.append(booleanvalue)
            combo_of_vals.append([value,booleanvalue,name,i])
            
        
        
    return(combo_of_vals)

Next I can determine the list of genes for each super enhancer at a set FDR threshold, for example at 0.1, this means every sample may have a different 'change in rank' cutoff value but they all will meet the FDR 0.1 cutoff criteria. 

I can iterate through changes in rank while maintaining a set FDR and keep all SEs and Genes that meet said threshold; I can then remove all duplicate rows based on SE name; keeping the last occurance (the smallest change in rank that met the FDR requirement). These cutoff values are vizualized below.

In [ ]:
matplotlib.pyplot.figure(figsize=(10,10))
sns.distplot(Dev_values_only,axlabel= 'Rank Order Changes')
matplotlib.pyplot.axvline(x=-2190, color='black')
matplotlib.pyplot.axvline(x=-2140, color='purple')
matplotlib.pyplot.axvline(x=-2100, color='blue')
matplotlib.pyplot.axvline(x=-2050, color='teal')
matplotlib.pyplot.axvline(x=-2000, color='green')
matplotlib.pyplot.axvline(x=-1950, color='cyan')
matplotlib.pyplot.axvline(x=-1900, color='yellow')
matplotlib.pyplot.axvline(x=-1800, color='red')


#we can zoom in using this 
matplotlib.pyplot.figure(figsize=(10,10))
sns.distplot(Dev_values_only,axlabel= 'Rank Order Changes')
matplotlib.pyplot.xlim(-2500,0)
matplotlib.pyplot.axvline(x=-2190, color='black')
matplotlib.pyplot.axvline(x=-2140, color='purple')
matplotlib.pyplot.axvline(x=-2100, color='blue')
matplotlib.pyplot.axvline(x=-2050, color='teal')
matplotlib.pyplot.axvline(x=-2000, color='green')
matplotlib.pyplot.axvline(x=-1950, color='cyan')
matplotlib.pyplot.axvline(x=-1900, color='yellow')
matplotlib.pyplot.axvline(x=-1800, color='red')


# Downregulated FDR cutoff analysis 

This iterates through various changes in rank and keeps genes that pass the FDR of 0.1 for each SE

In [ ]:
Overall_FDR_Passed_Sets = []
for thresh in [-2190,-2140,-2100,-2050,-2000,-1950,-1900,-1800]:
    FDR_Threshold_Up = []
    for val in DevNames:
        geneset = [val]
        #chromose = val.split('_')[-1]
        for i in range(len(OVCAR_dt[val])):
            value = OVCAR_dt[val][i]
            #median = DESEQ_dt['Median'][i]
            #set threshold 
            Threshold = thresh
            gene = OVCAR_dt['Gene'][i]
            #genechrom = DESEQ_dt['Chromosomes'][i]
            if value < Threshold :
                geneset.append(gene)
        FDR_Threshold_Up.append(geneset)

    norms = []
    non_norms = []
    for i in range(len(FDR_Threshold_Up)):
        lengths = len(FDR_Threshold_Up[i])-1
        if i == 0 or i == 67:
            norms.append(lengths)
            non_norms.append(lengths)
        else:
            non_norms.append(lengths)
    print(np.sum(norms),np.sum(non_norms))

    FDR_RES = FDR_Calculation_Threshold(norms,non_norms,.1)
    FDR_Passed_Set = []
    for i in range(len(FDR_RES)):
        boolval = FDR_RES[i][1]
        if boolval == True:
            FDR_Passed_Set.append(FDR_Threshold_Up[i])
    #FDR_Passed_Set_dt = pd.DataFrame(FDR_Passed_Set)
    
    for i in range(len(FDR_Passed_Set)):
        Overall_FDR_Passed_Sets.append(FDR_Passed_Set[i])

I can then save these outputs in a few different variables and look at them

In [ ]:
Overall_FDR_Passed_Sets_dt = pd.DataFrame(Overall_FDR_Passed_Sets)
Overall_FDR_Passed_Sets_dt[:10]

In [ ]:
Overall_FDR_Passed_Sets_dt=Overall_FDR_Passed_Sets_dt.drop_duplicates(subset=0, keep = 'last')
Overall_FDR_Passed_Sets_dt=Overall_FDR_Passed_Sets_dt.reset_index(drop='True')
Overall_FDR_Passed_Sets_dt[:10]

In [ ]:
OVT = np.array(Overall_FDR_Passed_Sets_dt)
OVTT = []
for row in OVT:
    res = list(filter(None, row))
    OVTT.append(res)
OVTT[:10]

I ran this to save the downregulated genes as an output file for later needs

In [ ]:
#Overall_FDR_Passed_Sets_dt.to_csv('OVCAR3_CRISPRI_SCREEN_FDR0.1_SEs_and_Genes.tsv', sep='\t', index= False)

I then repeat this for the upranked genes

In [ ]:
matplotlib.pyplot.figure(figsize=(10,10))
sns.distplot(Dev_values_only,axlabel= 'Rank Order Changes')
matplotlib.pyplot.axvline(x=2190, color='black')
matplotlib.pyplot.axvline(x=2140, color='purple')
matplotlib.pyplot.axvline(x=2100, color='blue')
matplotlib.pyplot.axvline(x=2050, color='teal')
matplotlib.pyplot.axvline(x=2000, color='green')
matplotlib.pyplot.axvline(x=1950, color='cyan')
matplotlib.pyplot.axvline(x=1900, color='yellow')


#we can zoom in using this 
matplotlib.pyplot.figure(figsize=(10,10))
sns.distplot(Dev_values_only,axlabel= 'Rank Order Changes')
matplotlib.pyplot.xlim(0,2500)
matplotlib.pyplot.axvline(x=2190, color='black')
matplotlib.pyplot.axvline(x=2140, color='purple')
matplotlib.pyplot.axvline(x=2100, color='blue')
matplotlib.pyplot.axvline(x=2050, color='teal')
matplotlib.pyplot.axvline(x=2000, color='green')
matplotlib.pyplot.axvline(x=1950, color='cyan')
matplotlib.pyplot.axvline(x=1900, color='yellow')

# Upregulated FDR cutoff analysis 

In [ ]:
Overall_Up_FDR_Passed_Sets = []
for thresh in [2190,2140,2100,2050,2000,1950,1900]:
    FDR_Threshold_Up = []
    for val in DevNames:
        geneset = [val]
        #chromose = val.split('_')[-1]
        for i in range(len(OVCAR_dt[val])):
            value = OVCAR_dt[val][i]
            #median = DESEQ_dt['Median'][i]
            #set threshold 
            Threshold = thresh
            gene = OVCAR_dt['Gene'][i]
            #genechrom = DESEQ_dt['Chromosomes'][i]
            if value > Threshold :
                geneset.append(gene)
        FDR_Threshold_Up.append(geneset)

    norms = []
    non_norms = []
    for i in range(len(FDR_Threshold_Up)):
        lengths = len(FDR_Threshold_Up[i])-1
        if i == 0 or i == 67:
            norms.append(lengths)
            non_norms.append(lengths)
        else:
            non_norms.append(lengths)
    print(np.sum(norms),np.sum(non_norms))

    FDR_RES = FDR_Calculation_Threshold(norms,non_norms,.1)
    FDR_Passed_Set = []
    for i in range(len(FDR_RES)):
        boolval = FDR_RES[i][1]
        if boolval == True:
            FDR_Passed_Set.append(FDR_Threshold_Up[i])
    #FDR_Passed_Set_dt = pd.DataFrame(FDR_Passed_Set)
    
    for i in range(len(FDR_Passed_Set)):
        Overall_Up_FDR_Passed_Sets.append(FDR_Passed_Set[i])

In [ ]:
Overall_Up_FDR_Passed_Sets_dt = pd.DataFrame(Overall_Up_FDR_Passed_Sets)
Overall_Up_FDR_Passed_Sets_dt[:10]

In [ ]:
Overall_Up_FDR_Passed_Sets_dt=Overall_Up_FDR_Passed_Sets_dt.drop_duplicates(subset=0, keep = 'last')
Overall_Up_FDR_Passed_Sets_dt=Overall_Up_FDR_Passed_Sets_dt.reset_index(drop='True')
Overall_Up_FDR_Passed_Sets_dt

In [ ]:
#Overall_Up_FDR_Passed_Sets_dt.to_csv('OVCAR3_CRISPRI_SCREEN_FDR0.1_SEs_and_Genes_UP.tsv', sep='\t', index= False)

In [ ]:
#upgenes =Overall_Up_FDR_Passed_Sets_dt[Overall_Up_FDR_Passed_Sets_dt[0] == 'OVCAR3_A3_60_chr20_rank_deviation_in_rank']
#upgenes

# Investigating all FDR .1 Up and Down Rank SEs and Genes 

First we create gene groups up and down

In [ ]:
genesetforclust = []
for i in range(len(Overall_FDR_Passed_Sets)):
    for j in range(len(Overall_FDR_Passed_Sets[i])):
        if j > 0:
            genesetforclust.append(Overall_FDR_Passed_Sets[i][j])
genesetdownarray = np.unique(genesetforclust)

In [ ]:
genesetforclustup = []
for i in range(len(Overall_Up_FDR_Passed_Sets)):
    for j in range(len(Overall_Up_FDR_Passed_Sets[i])):
        if j > 0:
            genesetforclustup.append(Overall_Up_FDR_Passed_Sets[i][j])
genesetuparray = np.unique(genesetforclustup)

In [ ]:
nwsetsarrayupdown = np.concatenate((genesetdownarray, genesetuparray), axis=0)
nwsetsarrayupdown[:10]

Make a DT for all genes that are in the updown group

In [ ]:
NEWDT = OVCAR_dt.loc[OVCAR_dt['Gene'].isin(nwsetsarrayupdown)]

In [ ]:
rnkanmescols = OVCAR_dt.columns[95:]
rnkanmescols[:10]

In [ ]:
rankcolnames = []
for row in OVCAR_dt.columns[1:194]:
    rankcolnames.append(row)

Retrieve all Ranks for the full dataset

In [ ]:
Rank_DESEQ_DT = OVCAR_dt.drop(rankcolnames,axis=1)
Rank_DESEQ_DT[:10]

In [ ]:
Rank_DESEQ_DT.shape

In [ ]:
rank_cols_names_for_ind = Rank_DESEQ_DT.pivot_table(index='Gene').columns
rank_cols_names_for_ind[:10]

In [ ]:
#Take only significant genes and rank changes 

This will make the DT just for the significant regions from the analysis 

In [ ]:
NEWDT_Rank = Rank_DESEQ_DT.loc[Rank_DESEQ_DT['Gene'].isin(nwsetsarrayupdown)]
NEWDT_Rank=NEWDT_Rank.reset_index(drop='True')
NEWDT_Rank.shape

In [ ]:
#NEWDT_Rank.to_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes.tsv', sep='\t', index= False)

If reading in this will make the same DT

In [ ]:
#NEWDT_Rank = pd.DataFrame.from_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes.tsv', sep = '\t')

In [ ]:
NEWDT_Rank.shape

In [ ]:
rank_cols_names_for_ind = NEWDT_Rank.pivot_table(index='Gene').columns
rank_cols_names_for_ind[:10]

## Full Datset of Genes 

First I need to determine the min rank as the clustering algorithm requires non-negative numbers, I will adjust all changes in rank by this value, then use a scaler to shrink them. The clustering will be done on both rows and columns and the gene changes in rank will be Z scored. This clustering and heatmap will look at the entire dataset of Genes and SEs. 

In [ ]:
maxes_of_ranks = []
for i in range(len(NEWDT_Rank.pivot_table(index='Gene').columns)):
    curcol = rank_cols_names_for_ind[i]
    maxes_of_ranks.append(np.min(NEWDT_Rank.pivot_table(index='Gene')[curcol]))
np.min(maxes_of_ranks)

In [ ]:
Rank_DESEQ_DT_Pivot=Rank_DESEQ_DT.pivot_table(index='Gene')

In [ ]:
for i in range(len(Rank_DESEQ_DT.pivot_table(index='Gene').columns)):
    curcol = rank_cols_names_for_ind[i]
    for j in range(len(Rank_DESEQ_DT_Pivot['OVCAR3_A10_87_chr6_rank_deviation_in_rank'])):
                       Rank_DESEQ_DT_Pivot[curcol][j] = Rank_DESEQ_DT_Pivot[curcol][j]+4322

for i in range(len(Rank_DESEQ_DT.pivot_table(index='Gene').columns)):
    curcol = rank_cols_names_for_ind[i]
    for j in range(len(Rank_DESEQ_DT_Pivot['OVCAR3_A10_87_chr6_rank_deviation_in_rank'])):
                       Rank_DESEQ_DT_Pivot[curcol][j] = Rank_DESEQ_DT_Pivot[curcol][j]/1000


In [ ]:
mapoclutrank =sns.clustermap(Rank_DESEQ_DT_Pivot,cmap='RdBu_r',robust=True,z_score=0, row_cluster=True,col_cluster=True,xticklabels = True,figsize=(20,10))
matplotlib.pyplot.xticks(rotation=30)
#matplotlib.pyplot.savefig('Rank_Heatmap_Wholeset.pdf', bbox_inches = 'tight')
matplotlib.pyplot.show()


## FDR Detected SEs and Genes

First I need to determine the min rank as the clustering algorithm requires non-negative numbers, I will adjust all changes in rank by this value, then use a scaler to shrink them. The clustering will be done on both rows and columns and the gene changes in rank will be Z scored. This clustering and heatmap will look at a subset of Genes and SEs, those that passed the .1 FDR threshold for increase or decreased rank. 

In [ ]:
NEWDT_Pivot=NEWDT_Rank.pivot_table(index='Gene')

for i in range(len(NEWDT_Pivot.pivot_table(index='Gene').columns)):
    curcol = rank_cols_names_for_ind[i]
    for j in range(len(NEWDT_Pivot['OVCAR3_A10_87_chr6_rank_deviation_in_rank'])):
                       NEWDT_Pivot[curcol][j] = NEWDT_Pivot[curcol][j]+4322
            
for i in range(len(NEWDT_Pivot.pivot_table(index='Gene').columns)):
    curcol = rank_cols_names_for_ind[i]
    for j in range(len(NEWDT_Pivot['OVCAR3_A10_87_chr6_rank_deviation_in_rank'])):
                       NEWDT_Pivot[curcol][j] = NEWDT_Pivot[curcol][j]/1000
NEWDT_Pivot[:10]

In [ ]:
zscoredata=scipy.stats.zscore(NEWDT_Pivot,axis=0)
zscoredata

In [ ]:
zdf = pd.DataFrame(zscoredata)
zdf.columns = NEWDT_Pivot.columns
zdf.index = NEWDT_Pivot.index
zdf

In [ ]:
Ks = range(1, 10)
km = [skcluster.KMeans(n_clusters=i) for i in Ks]
score = [km[i].fit(zdf).score(zdf) for i in range(len(km))]

In [ ]:
matplotlib.pyplot.plot(Ks,score)
#matplotlib.pyplot.savefig('Kmeans_Elbow_plot.pdf')

In [ ]:
#km=skcluster.KMeans(n_clusters=3).fit(NEWDT_Pivot)
#cluster_map_df = pd.DataFrame()
#cluster_map_df['data_index'] = NEWDT_Pivot.index.values
#cluster_map_df['cluster'] = km.labels_

In [ ]:
N = 3
km=skcluster.KMeans(n_clusters=N,n_init=50,max_iter=500,random_state=0).fit(zdf)
cluster_map_df = pd.DataFrame()
cluster_map_df['data_index'] = zdf.index.values
cluster_map_df['cluster'] = km.labels_
for i in range(0,N):
    idx=cluster_map_df['data_index'][cluster_map_df.cluster == i]
    idxgenes = []
    for row in idx:
        idxgenes.append(row)
    idxgenes[:10]
    Cluster0=zdf.loc[zdf.index.isin(idxgenes)]
    print(np.mean(Cluster0['OVCAR3_A3_60_chr20_rank_deviation_in_rank']),len(idxgenes))

In [ ]:
km=skcluster.KMeans(n_clusters=N,n_init=50,max_iter=500,random_state=0).fit(zdf)
cluster_map_df = pd.DataFrame()
cluster_map_df['data_index'] = zdf.index.values
cluster_map_df['cluster'] = km.labels_

In [ ]:
idx=cluster_map_df['data_index'][cluster_map_df.cluster == 0]
idxgenes = []
for row in idx:
    idxgenes.append(row)
idxgenes[:10]
Cluster0=zdf.loc[zdf.index.isin(idxgenes)]
print(np.mean(Cluster0['OVCAR3_A3_60_chr20_rank_deviation_in_rank']))

idx=cluster_map_df['data_index'][cluster_map_df.cluster == 1]
idxgenes = []
for row in idx:
    idxgenes.append(row)
idxgenes[:10]
Cluster1=zdf.loc[zdf.index.isin(idxgenes)]
print(np.mean(Cluster1['OVCAR3_A3_60_chr20_rank_deviation_in_rank']))

idx=cluster_map_df['data_index'][cluster_map_df.cluster == 2]
idxgenes = []
for row in idx:
    idxgenes.append(row)
idxgenes[:10]
Cluster2=zdf.loc[zdf.index.isin(idxgenes)]
print(np.mean(Cluster2['OVCAR3_A3_60_chr20_rank_deviation_in_rank']))

In [ ]:
Cluster0df=pd.DataFrame(Cluster0)
Cluster1df=pd.DataFrame(Cluster1)
Cluster2df=pd.DataFrame(Cluster2)

In [ ]:
#Cluster0df.to_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_Cluster0_mix60.tsv', sep='\t', index= True)
#Cluster1df.to_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_Cluster1_down60.tsv', sep='\t', index= True)
#Cluster2df.to_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_Cluster2_up60.tsv', sep='\t', index= True)

In [ ]:
#read in existing data 
#Cluster0df = pd.read_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_Cluster0_mix60.tsv', sep='\t')
#Cluster0df = Cluster0df.pivot_table(index='Gene')
#Cluster1df = pd.read_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_Cluster1_down60.tsv', sep ='\t')
#Cluster1df = Cluster1df.pivot_table(index='Gene')
#Cluster2df = pd.read_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_Cluster2_up60.tsv', sep ='\t')
#Cluster2df = Cluster2df.pivot_table(index='Gene')

I want to plot down, up, then mixed clusters in that order 

In [ ]:
merged_df = pd.concat([Cluster1df, Cluster2df, Cluster0df])
merged_df[:10]

In [ ]:
#merged_df.to_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_ZscoreRankChange_AllClust_downupmix60.tsv', sep='\t', index= True)

In [ ]:
mapoclutrank =sns.clustermap(merged_df,cmap='RdBu_r',z_score=0,robust=True, method= 'average', metric='euclidean', row_cluster=False, col_cluster=True,xticklabels = True,figsize=(20,10))
matplotlib.pyplot.xticks(rotation=30)
#matplotlib.pyplot.savefig('Rank_Heatmap_RankDownUP_FDR.1Set.kmeans3clusters_version2.pdf', bbox_inches = 'tight')
matplotlib.pyplot.show()

In [ ]:
#mapoclutrank.ax_heatmap.invert_xaxis()
#mapoclutrank.ax_col_dendrogram.invert_xaxis()
#mapoclutrank.fig

In [ ]:
#this method gets scr1 and dorm1 near eachother, they are close via average/euclidean but the dendrogram plots from high to low, this also has identifiable clusters 
mapoclutrank =sns.clustermap(merged_df,cmap='RdBu_r',z_score=0,robust=True, method= 'complete', metric='correlation', row_cluster=False, col_cluster=True,xticklabels = True,figsize=(20,10))
matplotlib.pyplot.xticks(rotation=30)
#matplotlib.pyplot.savefig('Rank_Heatmap_RankDownUP_FDR.1Set.kmeans3clusters_complete_correlation.pdf', bbox_inches = 'tight')
matplotlib.pyplot.show()


# Investigate General Patterns

I can now use this data to investigate general patterns and structures 

In [ ]:
No_dups_array = np.array(Overall_FDR_Passed_Sets_dt)
No_dups_array

In [ ]:
Cleaned_No_Dupe_Array = []
for i in range(len(No_dups_array)):
    filt=list(filter(None,No_dups_array[i]))
    Cleaned_No_Dupe_Array.append(filt)

In [ ]:
#pdCNA= pd.DataFrame(Cleaned_No_Dupe_Array)
#pdCNA.to_csv('FDR.1_Screen_Results_Downgenes_everySE.tsv',sep='\t')

In [ ]:
lengthsofsets = []
firsts = []
for i in range(len(Cleaned_No_Dupe_Array)):
    lengthsofsets.append(len(Cleaned_No_Dupe_Array[i]))
    firsts.append(Cleaned_No_Dupe_Array[i][0])

In [ ]:
matplotlib.pyplot.figure(figsize=(25,25))


objects = firsts
y_pos = np.arange(len(objects))
performance = lengthsofsets
matplotlib.pyplot.barh(y_pos, performance, align='center', alpha=0.5)
matplotlib.pyplot.yticks(y_pos, objects)
matplotlib.pyplot.xlabel('Number of Genes Downranked')
matplotlib.pyplot.title('Number of Genes Downranked per SE')
#matplotlib.pyplot.savefig('Bar_Chart_Genes_Per_SE_FDR.1_DR.pdf', bbox_inches = 'tight')


matplotlib.pyplot.show()

In [ ]:
namesdt = pd.DataFrame(firsts)
namesdt['nameofSE'] = firsts
namesdt['num_down_genes'] = performance
namesdt[:10]

Create a DF for plotting purposes with names and downregulated genes 

In [ ]:
sortred_names_dt=namesdt.sort_values('num_down_genes')
sortred_names_dt[:10]

In [ ]:
matplotlib.pyplot.figure(figsize=(25,25))


objects = sortred_names_dt['nameofSE']
y_pos = np.arange(len(objects))
performance = sortred_names_dt['num_down_genes']
matplotlib.pyplot.barh(y_pos, performance, align='center', alpha=0.5)
matplotlib.pyplot.yticks(y_pos, objects)
matplotlib.pyplot.xlabel('Number of Genes Downranked')
matplotlib.pyplot.title('Number of Genes Downranked per SE')
#matplotlib.pyplot.savefig('Bar_Chart_Genes_Per_SE_FDR.1_DR_SORTED.pdf', bbox_inches = 'tight')


matplotlib.pyplot.show()

Read in the size information and construct the size and singnal dataframe 

In [ ]:
OVCAR_SE_SIZEDT=pd.read_csv("/Users/mkelly95/Documents/IGV/HG19/H3K27ac_Signal_over_BRD4_SuperEnhancers_TPM_1bpovlp.bed", sep="\t",header = None)
OVCAR_SE_SIZEDT[:10]

In [ ]:
serankspots = []
for i in range(len(OVTT)):
    serankspots.append((str.split(OVTT[i][0],'_')[2],str.split(OVTT[i][0],'_')[3]))
serankspots[:10]

In [ ]:
sizesforchrscheck = []
for i in range(len(OVCAR_SE_SIZEDT[0])):
    size = OVCAR_SE_SIZEDT[2][i] - OVCAR_SE_SIZEDT[1][i]
    sizesforchrscheck.append(size)
sizesforchrscheck[:10]

In [ ]:
stuffhelp = []
for row in serankspots:
    if len(row[0]) < 4 and row[0][0] != 'F':
        stuffhelp.append(int(row[0]))    
stuffhelp[:10]

In [ ]:
sizegrab = []
for row in stuffhelp:
    i = row-1
    sizegrab.append(sizesforchrscheck[i])
sizegrab[:10]

In [ ]:
plot_mat_names = []
plot_mat_lengths = []
plot_mat_numgenes = []
plot_mat_rank = []
for i in range(len(sortred_names_dt['nameofSE'])):
    name = sortred_names_dt['nameofSE'][i]
    numberofgenes = sortred_names_dt['num_down_genes'][i]
    splitname = str.split(name,'_')
    num = splitname[2]
    for j in range(len(stuffhelp)):
        if str(stuffhelp[j]) == num:
            plot_mat_names.append(name)
            plot_mat_lengths.append(int(sizegrab[j]))
            plot_mat_numgenes.append(int(numberofgenes))
            plot_mat_rank.append(stuffhelp[j])
            #print(name, numberofgenes, stuffhelp[j],sizegrab[j])
plottingdt = pd.DataFrame(plot_mat_names)
plottingdt['names'] = plot_mat_names
plottingdt['NumGenesDown'] = plot_mat_numgenes
plottingdt['SELength'] = plot_mat_lengths
plottingdt['SERank_H3K27ac'] = plot_mat_rank
plottingdt[:10]

In [ ]:
matplotlib.pyplot.figure(figsize=(25,25))


objects = plottingdt['NumGenesDown']
y_pos = np.arange(len(objects))
performance = plottingdt['SERank_H3K27ac']
matplotlib.pyplot.scatter(performance, objects, s=150)
matplotlib.pyplot.xlabel('H3K27ac Rank of SE')
matplotlib.pyplot.ylabel('Number of Genes')
matplotlib.pyplot.title('H3K27ac Rank vs Counts')
#matplotlib.pyplot.savefig('SE_rank_vs_Genes_FDR.1.pdf', bbox_inches = 'tight')
matplotlib.pyplot.show()

In [ ]:
matplotlib.pyplot.figure(figsize=(25,25))


objects = plottingdt['NumGenesDown']
y_pos = np.arange(len(objects))
performance = plottingdt['SELength']
matplotlib.pyplot.scatter(performance, objects, s=150)
matplotlib.pyplot.xlabel('Size of SE (bp)')
matplotlib.pyplot.ylabel('Number of Genes')
matplotlib.pyplot.title('SE Size vs Counts')
#matplotlib.pyplot.savefig('SE_Size_vs_Genes_FDR.1.pdf', bbox_inches = 'tight')
matplotlib.pyplot.show()

# Fold Change

I now want to investigate the similarities between the Rank Change and Log Fold Change data; as we do not have replicate's we can't call DEGS with LFCs as we can't estimate the variation in the data. The list for DEGs will be the same as determined by the Rank Change analysis but the values will be LFCs, they will be calculated across the whole dataset similarly to how it was done with the rank changes. The starting dataset is a little different, instead of using the VST transformed DESEQ dataset I am starting with the raw counts data and performing the log2/fold change calculations on that. 

In [ ]:
#read in the data and remove the two outlier samples we earlier removed in the DESEQ normalization for rank based analysis
OVCAR_dt = pd.read_csv("OVCAR3_96_Well_Screen_LowExpressed_Genes_Removed.tsv", sep="\t",header = 0)
OVCAR_dt[:10]
OVCAR_dt=OVCAR_dt.drop(axis =1, columns='OVCAR3_H7_61')
OVCAR_dt=OVCAR_dt.drop(axis =1, columns='OVCAR3_F2_64')
samplenames = []
for val in OVCAR_dt.columns:
    if val[0:3] == 'OVC':
        samplenames.append(val)
samplenames[:10]

In [ ]:
#add 1, convert to tpm
OVCAR2_dt=pd.read_csv("OVCAR3_96_Well_Screen_LowExpressed_Genes_Removed.tsv", sep="\t",header = 0)
OVCAR2_dt=OVCAR2_dt.drop(axis =1, columns='Gene')
OVCAR2_dt=OVCAR2_dt.drop(axis =1, columns='OVCAR3_H7_61')
OVCAR2_dt=OVCAR2_dt.drop(axis =1, columns='OVCAR3_F2_64')

OVCAR2_dt = OVCAR2_dt+1
for val in samplenames:
    OVCAR2_dt[val] = OVCAR2_dt[val]/(np.sum(OVCAR2_dt[val]/1000000))
#OVCAR2_dt = np.log2(OVCAR2_dt)
OVCAR2_dt[:10]

In [ ]:
for val in samplenames:
    OVCAR_dt[val] = OVCAR2_dt[val]
OVCAR_dt[:10]

In [ ]:
#dorm and scramble 
#samplenames[0]
#samplenames[67]

In [ ]:
OVCAR_nozero = np.array(OVCAR_dt)

In [ ]:
genemean = []
for row in OVCAR_nozero[:,1:]:
    means = np.median(row)
    genemean.append(means)
OVCAR_dt['Gene_Mean'] = genemean


In [ ]:
#start after the mean is taken
#need raw counts at start 
OVCAR_dt[1:10]

In [ ]:
DevNames = []
#Deviation_in_Rank =[]
Deviation_in_Fold = []
for val in samplenames:
    geneset = [val]
    newname = val+'_LFC'
    DevNames.append(newname)
    for i in range(len(OVCAR_dt[val])):
        rank = OVCAR_dt[val][i]
        sample = val
        #median = DESEQ_dt['Median'][i]
        true_rank = OVCAR_dt['Gene_Mean'][i]
        gene = OVCAR_dt['Gene'][i]
        dif = rank/true_rank
        #genechrom = DESEQ_dt['Chromosomes'][i]
        LFC_dif = np.log2(dif)
        #OVCAR_dt[newname][i] = true_rank
        Deviation_in_Fold.append([sample, gene, LFC_dif])

In [ ]:
for val in DevNames:
    set_of_values = []
    for i in range(len(Deviation_in_Fold)):
        name = val.split('_')[:3]
        name2 = Deviation_in_Fold[i][0].split('_')[:3]
        vals = Deviation_in_Fold[i][2]
        if name == name2:
            set_of_values.append(vals)
    OVCAR_dt[val] = set_of_values
OVCAR_dt[:10]

In [ ]:
LFC_only_dt=OVCAR_dt[OVCAR_dt.columns[96:]]
LFC_only_dt['Gene'] = OVCAR_dt['Gene']
LFC_only_dt[:10]

In [ ]:
ControlNames = ['OVCAR3_H10_DORM_1_LFC','OVCAR3_G11_SCRAMBLE_1_LFC']

In [ ]:
Dev_values_control = []
maxes = []
mins = []
for name in ControlNames:
    namevals = []
    for i in range(len(OVCAR_dt[name])):
        Dev_values_control.append(OVCAR_dt[name][i])
        namevals.append(OVCAR_dt[name][i])
    maxes.append(np.max(namevals))
    mins.append(np.min(namevals))
Dev_values_control[:3]
Dev_values_only = []
maxes = []
mins = []
for name in DevNames:
    namevals = []
    for i in range(len(OVCAR_dt[name])):
        Dev_values_only.append(OVCAR_dt[name][i])
        namevals.append(OVCAR_dt[name][i])
    maxes.append(np.max(namevals))
    mins.append(np.min(namevals))
Dev_values_only[:3]

In [ ]:
GenevalsDT = LFC_only_dt.loc[LFC_only_dt['Gene'].isin(nwsetsarrayupdown)]
GenevalsDT=GenevalsDT.reset_index(drop='True')
GenevalsDT.shape

In [ ]:
JustLFCSdt=GenevalsDT
LFC_Pivot=JustLFCSdt.pivot_table(index='Gene')

Also make for Rank change 

In [ ]:
JustRanksdt=NEWDT_Rank[NEWDT_Rank.columns]
JR_Pivot=NEWDT_Rank.pivot_table(index='Gene')

Then I can calculate the correlation between the LFC/Rank Cahnge values for the genes in each super enhancer 

In [ ]:
names1 = JR_Pivot.columns
names2= LFC_Pivot.columns
Correlations_All_Cols = []
for i in range(len(names1)):
    corval = JR_Pivot[names1[i]].corr(LFC_Pivot[names2[i]])
    Correlations_All_Cols.append(corval)


This is the average correlation between LFC and RC

In [ ]:
np.mean(Correlations_All_Cols)

This is a plot of all correlation values, marked are the average (Blue), SE60 (black), and SE14 (Purple)

In [ ]:
matplotlib.pyplot.scatter(x=Correlations_All_Cols, y=[1]*94, c=Correlations_All_Cols, cmap='Reds')
#SE60
matplotlib.pyplot.axvline(x=.8713, color='black')
#SE14
matplotlib.pyplot.axvline(x=.947, color='purple')
#AVG
matplotlib.pyplot.axvline(x=.79, color='blue')
matplotlib.pyplot.xlabel('Correlation Between LFC and Rank Change for Sig Genes')
#matplotlib.pyplot.xticks(ticks=Correlations_All_Cols, labels=LFC_only_dt.columns)
#matplotlib.pyplot.savefig('Correlation_of_all_sig_Genes_RC_vs_LFC_Allmarked_bluemed_black60_pur14.pdf', bbox_inches = 'tight')

I am next going to create a subset of this data where we just look at SE60's significant genes for plotting purposes

In [ ]:
#for SE60 only plotting 
nwsetnwsetsarrayupdown = pd.read_csv('SE60_Genes_FDR.1_up_down_genes.txt',sep = '\n')

#forjustse60
NEWDT_Rank60 = NEWDT.loc[Rank_DESEQ_DT['Gene'].isin(nwsetnwsetsarrayupdown['OVCAR3_A3_60_chr20_rank_deviation_in_rank'])]
NEWDT_Rank60=NEWDT_Rank60.reset_index(drop='True')
NEWDT_Rank60.shape

GenevalsDT60 = GenevalsDT.loc[GenevalsDT['Gene'].isin(nwsetnwsetsarrayupdown['OVCAR3_A3_60_chr20_rank_deviation_in_rank'])]
GenevalsDT60=GenevalsDT60.reset_index(drop='True')
GenevalsDT60.shape



In [ ]:
JRC60 =NEWDT_Rank60.pivot_table(index='Gene')
LFCC60 = GenevalsDT60.pivot_table(index='Gene')

I can next plot, for every super-enhancer, or in this case SE60 then SE14 the Rank Change (X axis) vs the LFC (Y axis) for every gene determined as significant by the FDR .1 approach in any SE

In [ ]:
#SE60
#all genes corr = 0.88 
#just SE genes sig corr = 0.94 
matplotlib.pyplot.scatter(x=JR_Pivot['OVCAR3_A3_60_chr20_rank_deviation_in_rank'],y=LFC_Pivot['OVCAR3_A3_60_LFC'])
#matplotlib.pyplot.savefig('Correlation_of_all_sig_Genes_RC_vs_LFC_SE60.pdf', bbox_inches = 'tight')

In [ ]:
#SE14
matplotlib.pyplot.scatter(x=JR_Pivot['OVCAR3_D7_14_chr1_rank_deviation_in_rank'],y=LFC_Pivot['OVCAR3_D7_14_LFC'])
#matplotlib.pyplot.savefig('Correlation_of_all_sig_Genes_RC_vs_LFC_SE14.pdf', bbox_inches = 'tight')

This plot shows all FDR .1 genes from all samples in blue and the SE60 FDR genes in red, with the values being Rank Change (X axis) and LFC change (Y axis) for SE60

In [ ]:
ax = matplotlib.pyplot.gca()


ax.scatter(x=JR_Pivot['OVCAR3_A3_60_chr20_rank_deviation_in_rank'],y=LFC_Pivot['OVCAR3_A3_60_LFC'], color="b")

ax.scatter(x=JRC60['OVCAR3_A3_60_chr20_rank_deviation_in_rank'],y=LFCC60['OVCAR3_A3_60_LFC'], color="r")

#matplotlib.pyplot.savefig('Correlation_of_sig_Genes_RC_vs_LFC_SE60_allgenesblue_se60onlyred.pdf', bbox_inches = 'tight')


In [ ]:
#read it in 
#GenevalsDT = pd.read_csv('OVCAR3_CRISPRI_SCREEN_DESEQ2_Normalized_FDR.1SigRankChangeGenes_LFC_Values_95cols_medianallsamples.tsv', sep ='\t')

In [ ]:
combodf = pd.concat([JR_Pivot,LFC_Pivot],axis=1)
combodf[:10]

This html file contains the correlation of all RC to RC and LFC values, the main takeaway is that for every sample the highest correlation is between the RC and LFC values for that same SE.

In [ ]:
corrtype = combodf.corr()
corrtype.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
htmlname =corrtype.style.background_gradient(cmap='coolwarm').set_precision(2).render()


In [ ]:
#html_file = open("FDR.1GenesRCvsLFC_Corrs.html","w")
#html_file.write(htmlname)
#html_file.close()

In [ ]:
from scipy.spatial import distance
from scipy.cluster import hierarchy

correlations = combodf.corr()
correlations_array = np.asarray(combodf.corr())

row_linkage = hierarchy.linkage(
    distance.pdist(correlations_array), method='average')

col_linkage = hierarchy.linkage(
    distance.pdist(correlations_array.T), method='average')

sns.clustermap(correlations, row_linkage=row_linkage, col_linkage=col_linkage, method="average", figsize=(20, 20))

In [ ]:
ranksonlynamesheatmap=[]
for name in correlations.columns:
    if 'rank_deviation_in_rank' in name:
        ranksonlynamesheatmap.append(name)

In [ ]:
#correlations[94:][ranksonlynamesheatmap]

In [ ]:
datplot=sns.clustermap(correlations[94:][ranksonlynamesheatmap],cmap='coolwarm', figsize=(20, 20))

In [ ]:
datplot=sns.clustermap(correlations[94:][ranksonlynamesheatmap],row_cluster=False,col_cluster=False,cmap='coolwarm', figsize=(20, 20))
#matplotlib.pyplot.savefig('LFC_VS_RC_Corrplot_UnClustered.pdf')